
## IMPORTS


In [1]:
from vpython import *

<IPython.core.display.Javascript object>


## NET/MODULE/EVENT CLASS DEFINTIONS 
### All classes used in this project are defined in this block plus helper functions to run a clock signal 


In [2]:
#####################5000#########################################    
                    ## HELPER FUCNTION ## 
def clock(val, obj): 
    if val == 0: 
        if obj.value == "X":
            obj.value = "0"
        elif obj.value == "0":
            obj.value = "1"
        elif obj.value == "1":
            obj.value = "0"
##############################################################    

##############################################################    
######################### "NET" CLASS ########################
class net: 
    __signal_types = ["0", "1", "X"]
    
    def __init__(self, id, val="X"): 
        self.driver = None
        self.receivers = []
        self.id = id
        self.prev_value = val
        self.value = val
        self.data = []
            
    def update(self): 
        if self.value == "X": 
            self.data.append(0.5)
        else: 
            self.data.append(int(self.value))
##############################################################    
##############################################################    
     
##############################################################    
######################## "GATE" CLASS ########################
class module: 
    __gate_types = ["NAND", "NOR", "XOR", "OR", "AND"]
    __signal_types = ["0", "1", "X"]

    def __init__(self, input, output, func, id, dt=1): 
        ## Time for delay ##
        self.time = 0
        self.dt = dt 
        
        self.delay = 5 ## delay in nanoseconds 
        self.q = [] ## "FIFO" queue but as a list 
        
        ## HOLD THE STATE OF THE MODULE ## 
        self.valid = True
        
        ## VALIDATE MODULE INPUTS/FUNCTION ## (not-important)
        if not( func.upper() in module.__gate_types ): 
            print("Gate function not found.")
            self.valid = False
        
        if len(input) != 2:
            print("Invalid input net size.")
            self.valid = False
            
        if not( input[0].value in module.__signal_types ) or not( input[1].value in module.__signal_types):
            print("Invalid input type.")
            self.valid = False
        
        ## IMPORTANT ##
        self.id = id
        self.inputs = input ## must be an array with 2 net elements( 2 inputs ) 
        self.output = output ## net 
        self.func = func
        self.event()

        for net in self.inputs: 
            net.receivers.append(self)
            
    def event(self):        
        e = {} ## OBJECT TO HOLD THE EVENT
        
        ## OUTPUT BASED ON FUNCTION ## 
        if self.inputs[0].value == "X" or self.inputs[1].value == "X":
            e["output"] = "X"
        elif self.func == "NAND": 
            val = not(int(self.inputs[0].value) and int(self.inputs[1].value))
            e["output"] = str(int(val))
        elif self.func == "NOR":
            val = not(int(self.inputs[0].value) or int(self.inputs[1].value))
            e["output"] = str(int(val))
        elif self.func == "AND": 
            val = (int(self.inputs[0].value) and int(self.inputs[1].value))
            e["output"] = str(int(val))
        elif self.func == "OR":
            val = (int(self.inputs[0].value) or int(self.inputs[1].value))
            e["output"] = str(int(val))
        elif self.func == "XOR":
            val = (int(self.inputs[0].value) ^ int(self.inputs[1].value))
            e["output"] = str(int(val))
            
        ## DELAY EVENT ## 
        e["trigger"] = self.time + self.delay 
        self.q.append(e)
    
    def update(self):  
        if len(self.q) != 0:     
            ## CHECK IF THE DELAY TIME HAS BEEN REACHED ## 
            for i,obj in enumerate(self.q): 
                if self.time >= obj["trigger"]:
                    self.output.value = obj["output"]
                    self.q.pop(i) ## REMOVE THE EVENT IF IT HAPPENED ##
                    
        self.time += self.dt

##############################################################
##############################################################


##############################################################    
######################## "EVENT" CLASS #######################
class event: 
    def __init__(self, nets, gates, dt=1): 
        self.nets = nets ## should be an array/list 
        self.gates = gates
        self.time = 0
        self.data = [] ## time step value s
        self.dt = dt
        
    def listen(self):
        for i in self.nets: 
            if i.value != i.prev_value: 
                ## que event
                for j in i.receivers:
                    j.event()
            i.prev_value = i.value
            i.update()

        self.data.append(self.time)
        self.time += self.dt 
        
    def update(self):
        for gate in self.gates: 
            gate.update()
##############################################################
##############################################################            

In [3]:
class fullAdder: 
    def __init__(self, A, B, C_in): 
        ## INTERMEDIATE NETS ## 
        self.xor1_out = net( id='0')
        self.and1_out = net( id='1')
        self.and2_out = net( id='2')
        
        ## OUTPUT NETS ## 
        self.S = net( id='3')
        self.C_out = net( id='4')
        
        ## SAVE NETS ##
        self.nets=[self.xor1_out, self.and1_out, self.and2_out, self.S, self.C_out]
        
        self.xor1_gate = module( input=[A, B], 
                            output=self.xor1_out,
                            func="XOR",
                            id='1' )
        
        self.xor2_gate = module( input=[C_in, self.xor1_out], 
                            output=self.S,
                            func="XOR",
                            id='2' )
        
        self.and1_gate = module( input=[self.xor1_out, C_in], 
                             output=self.and1_out, 
                             func="AND", 
                             id='3' ) 
        
        self.and2_gate = module( input=[A, B],
                             output=self.and2_out, 
                             func="AND", 
                             id='4' )
        
        self.or_gate = module( input=[self.and1_out, self.and2_out],
                          output=self.C_out, 
                          func="OR",
                          id='5' ) 
        
        ## SAVE OR GATES ## 
        self.gates=[self.xor1_gate, self.xor2_gate, self.and1_gate, self.and2_gate, self.or_gate]

In [4]:
scene = canvas()

## 4 BIT ADDER ## 
A = [net( id=f"a{i}") for i in range(4)]
B = [net( id=f"b{i}") for i in range(4)]
C_in = net( id="c0")
f0=fullAdder(A[0], B[0], C_in)
f1=fullAdder(A[1], B[1], f0.C_out)
f2=fullAdder(A[2], B[2], f1.C_out)
f3=fullAdder(A[3], B[3], f2.C_out) 
adders=[f0,f1,f2,f3]
 
S = [ adders[i].S for i in range(len(adders)) ]
sys=event( nets=A+B+[C_in]+[net for i in adders for net in i.nets],
           gates=[gate for i in adders for gate in i.gates]) 

C_in.value = "0"

a_str = "1110"
b_str = "0111"
for i in range(len(a_str)): 
    A[i].value = a_str[len(a_str)-i-1]
for i in range(len(B)): 
    B[i].value = b_str[len(b_str)-i-1]


while sys.time < 500: 
    sys.listen()
    sys.update()

pl1 = gcurve(color=color.red)
pl2 = gcurve(color=color.cyan)

for i in range(len(sys.data)):
    pl1.plot(sys.data[i], S[0].data[i])
    
# print("S: " + S[3].value 
#       + S[2].value 
#       + S[1].value 
#       + S[0].value )
# print("C: " + adders[3].C_out.value )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1



# Refrence


In [5]:
scene = canvas()

## NETS
## DATA LINE AND CLOCK ## 
clk = net(val="0", id='1')
data = net(val="1", id='2') 
not_data = net(val="0", id='7')

## INTERMEDIATE ##
net3 = net(val="0", id='3')
net4 = net(val="0", id='4')
# print(len(S[i].data))
# print(len(sys.data))
## OUTPUTS ## 
Q = net(val="0", id='5')
Qnot = net(val="0", id='6')


## GATES 
not_gate = module( input=[data, data],
                output=not_data, 
                func="NAND", 
                id='5')

gate1 = module( input=[data, clk],
                  output=net3,
                  func="NAND",
                  id='1')

gate2 = module( input=[clk, not_data],
                  output=net4,
                  func="NAND",
                  id='2')

gate3 = module( input=[net3, Qnot],
                  output=Q,
                  func="NAND",
                  id='3')

gate4 = module( input=[Q, net4],
                  output=Qnot,
                  func="NAND",
                  id='4' )


## CONSOLIDATING ALL GATES AND NETS ## 
gates = [not_gate, gate1, gate2, gate3, gate4]
nets = [clk, data, not_data, net3, net4, Q, Qnot]
sys = event(nets=nets, gates=gates)

## RUN CIRCUIT FOR 400 TIME UNITS ## 
while sys.time <= 400: 
    sys.listen()
    clock(sys.time%20, clk)
    if sys.time == 200: 
        data.value = "0"
    sys.update()
    
## GRAPHING ## 
## GRAPH INPUT CLK
graph(ymin=-0.05, ymax=1.05)
n1 = gcurve(color=color.blue, label="CLOCK (driver)")
for i in range(len(sys.data)):
    n1.plot(sys.data[i], clk.data[i])

## GRAPH INPUT DATA
graph(ymin=-0.05, ymax=1.05)
n2 = gcurve(color=color.black, label="DATA (driver)")
for i in range(len(sys.data)):
    n2.plot(sys.data[i], data.data[i])
    
## GRAPH THE OUTPUT Q
graph(ymin=-0.05, ymax=1.05)
n5 = gcurve(color=color.red, label="Q (output)")
for i in range(len(sys.data)): 
    n5.plot(sys.data[i], Q.data[i])

## GRAPH THE OUTPUT ~Q
graph(ymin=-0.05, ymax=1.05)
n6 = gcurve(color=color.red, label="~Q (output)")
for i in range(len(sys.data)): 
    n6.plot(sys.data[i], Qnot.data[i])
    

## GRAPH INTERMEDIATE ## 
graph(ymin=-0.05, ymax=1.05)
n3 = gcurve(color=color.green, label="net3 (intermediate)")
for i in range(len(sys.data)): 
    n3.plot(sys.data[i], net3.data[i])

graph(ymin=-0.05, ymax=1.05)
n4 = gcurve(color=color.green, label="net4 (intermediate)")
for i in range(len(sys.data)): 
    n4.plot(sys.data[i], net4.data[i])


<IPython.core.display.Javascript object>